<a href="https://colab.research.google.com/github/anishgupta02/vectordb/blob/main/chromadb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install chromadb openai langchain tiktoken

In [2]:
'https://www.dropbox.com/s/vs6ocyvpzzncvwh/new_articles.zip'

'https://www.dropbox.com/s/vs6ocyvpzzncvwh/new_articles.zip'

In [3]:
import os
os.environ['OPENAI_API_KEY']='Give_your_key'

In [4]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader

In [5]:
#loading the data
loader = DirectoryLoader(r"/content/new_articles", glob="./*.txt")

In [6]:
document=loader.load()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [7]:
#chunking the data
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
text=text_splitter.split_documents(document)


In [8]:
len(text)

233

In [9]:
#creating database
from langchain import embeddings

In [10]:
persist_directory='db'

In [11]:
embedding=OpenAIEmbeddings()


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [12]:
import chromadb

In [16]:
vectordb=Chroma.from_documents(
    documents=text,
    embedding=embedding,
    persist_directory=persist_directory
)

In [17]:
#persist the db to disk
vectordb.persist()

In [18]:
vectordb=None

In [21]:
#Now we can load the persisted database from disk,and use it as normal
vectordb=Chroma(persist_directory=persist_directory,embedding_function=embedding)

In [22]:
vectordb

In [27]:
#Make retrieval
retriever1=vectordb.as_retriever()#give possible number of outputs

In [28]:
retriever2=vectordb.as_retriever(search_kwargs={'k':2})#gives only two outputs

In [29]:
docs1=retriever1.get_relevant_documents('how much money did Microsoft raise? ')

In [36]:
docs1

[Document(page_content='April 28, 2023\n\nVC firms including Sequoia Capital, Andreessen Horowitz, Thrive and K2 Global are picking up new shares, according to documents seen by TechCrunch. A source tells us Founders Fund is also investing. Altogether the VCs have put in just over $300 million at a valuation of $27 billion to $29 billion. This is separate to a big investment from Microsoft announced earlier this year, a person familiar with the development told TechCrunch, which closed in January. The size of Microsoft’s investment is believed to be around $10 billion, a figure we confirmed with our source.\n\nApril 25, 2023\n\nCalled ChatGPT Business, OpenAI describes the forthcoming offering as “for professionals who need more control over their data as well as enterprises seeking to manage their end users.”', metadata={'source': '/content/new_articles/05-03-chatgpt-everything-you-need-to-know-about-the-ai-powered-chatbot.txt'}),
 Document(page_content='April 28, 2023\n\nVC firms inc

In [31]:
len(docs1)

4

In [32]:
docs2=retriever2.get_relevant_documents('how much money did Microsoft raise? ')

In [37]:
docs2

[Document(page_content='April 28, 2023\n\nVC firms including Sequoia Capital, Andreessen Horowitz, Thrive and K2 Global are picking up new shares, according to documents seen by TechCrunch. A source tells us Founders Fund is also investing. Altogether the VCs have put in just over $300 million at a valuation of $27 billion to $29 billion. This is separate to a big investment from Microsoft announced earlier this year, a person familiar with the development told TechCrunch, which closed in January. The size of Microsoft’s investment is believed to be around $10 billion, a figure we confirmed with our source.\n\nApril 25, 2023\n\nCalled ChatGPT Business, OpenAI describes the forthcoming offering as “for professionals who need more control over their data as well as enterprises seeking to manage their end users.”', metadata={'source': '/content/new_articles/05-03-chatgpt-everything-you-need-to-know-about-the-ai-powered-chatbot.txt'}),
 Document(page_content='April 28, 2023\n\nVC firms inc

In [33]:
len(docs2)

2

In [38]:
#make a chain
from langchain.chains import RetrievalQA

In [40]:
llm=OpenAI()

In [75]:
qa_chain=RetrievalQA.from_chain_type(
    llm=OpenAI(),
    chain_type='stuff',
    retriever=retriever1
)

In [75]:
# Full example
query = "How much money did Microsoft raise?"


In [76]:
llm_response = qa_chain(query)


In [77]:
print(llm_response)

{'query': 'How much money did Microsoft raise?', 'result': " The size of Microsoft's investment was believed to be around $10 billion."}


In [75]:
#to delete the db

In [ ]:
'''
!zip -r db.zip ./db
'''

In [ ]:
'''
vectordb.delete_collection()
vectordb.persist()
'''

In [ ]:
'''
!rm -rf db/
'''